# Student name and e-mail

name : 

e-mail : 

In [ ]:
USE_GOOGLE_COLAB = False
MY_GOOGLE_DRIVE_PATH = 'TP_Modeling' # where files will be stored

In [ ]:
if USE_GOOGLE_COLAB:
  from google.colab import drive
  from os.path import join 
  import os
  ROOT = "/content/drive/"     # default location for the drive

  PROJECT_PATH = join(ROOT, 'MyDrive', MY_GOOGLE_DRIVE_PATH)
  BASE_PATH = join(ROOT, 'MyDrive')
  
  drive.mount(ROOT, force_remount=False)           # we mount the google drive at /content/drive

  os.chdir(BASE_PATH)

  try:
    os.mkdir(PROJECT_PATH)
  except FileExistsError:
    pass
  os.chdir(PROJECT_PATH)

  ! git clone https://github.com/GFuhr/MF_FCM6.git
  
  os.chdir('./MF_FCM6/')
  ! git pull origin master
  os.chdir('./python')

  print('current folder')
  print(os.getcwd())

# python modules

In [ ]:
%load_ext cython

In [ ]:

from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation

import os
import cython
import datetime
import numpy as np
try:
    import yaml
    HAS_YAML=True
    display('Good News, YAML format accepted')
except ImportError:
    import json as js
    HAS_YAML=False
    display('only json accepted')

%matplotlib inline


# function used to generate animations and compute computation time

# Simulation kernel
Section where the main simulation code is compiled and loaded. A *run* folder will be created and used to store all the simulations output

In [ ]:
os.getcwd()
folder = os.path.split(os.getcwd())
if ('h2d' in folder) or ('advdiff' in folder):
    try:
        os.mkdir('../run')
    except FileExistsError:
        pass
    os.chdir('../run')
elif ('run' in folder):
    pass
elif ('python' in folder):
    try:
        os.mkdir('./run')
    except FileExistsError:
        pass
    os.chdir('./run')
    
display(HTML('<font size=4> data saved in the ./run subfolder </font>'))

In [ ]:
%%cython
import numpy as np
import pyximport

pyximport.install(reload_support=True, setup_args = {'include_dirs': np.get_include()})

In [ ]:
import importlib
os.chdir('../advdiff')
try: 
    importlib.reload(advdiff); import advdiff
except NameError:
    import advdiff
os.chdir('../run')

display(HTML('AdvDiff imported in current notebook'))

# Additional functions

<font size=3>
    Functions defined here have been made to simplify representation of outputs. These functions are not part of the labs and can be used directly
</font>

In [ ]:
from utils.timer import Timer
from utils.plotting import plot_results, plot_profile, figformat

In [ ]:
from utils.file import save_outputs

def save_with_params(output, params):
    prefix = ''
    if params.get('C', None) is not None:
        prefix = 'diff_'
    if params.get('V', None) is not None:
        prefix += 'adv_'
    prefix += '{0:03d}_'.format(params['Nx'])
    save_outputs(output, prefix=prefix)

In [ ]:
def load_from_string(params_str:str):
    """
    function used to load parameters from a string and load it into a dict used by simulations
    return : dict or ValueError if scheme is not correct
    """
    if not HAS_YAML:
        data = js.loads(params_str)
    else:
        data = yaml.safe_load (params_str)
    data['V'] = np.double(data['V'])
    try:
        data['Nx'] = int(data['Nx'])
    except TypeError:
        raise ValueError('Nx Value missing or not written as a number')
    try:
        data['Ny'] = int(data['Ny'])
    except TypeError:
        raise ValueError('Ny Value missing or not written as a number')
    except KeyError:
        print('Ny key not found')
    data['scheme'] = data['scheme'].lower().strip()
    if data['scheme'] not in ('eule', 'euli', 'rk2', 'rk4', 'cn'):
        raise ValueError('unknown time scheme')
    return data

In [ ]:
# function extracting the max value and his position for each serie of a list of profiles
def extract_max(Frames):
    list_max = np.zeros(shape=(2, len(Frames)))
    idx = 0
    for frame in Frames:
        list_max[:,idx] = (frame.argmax(), frame.max())
        idx += 1
    return np.asarray(list_max)

In [ ]:
# compute Fourier transform for a given field
def fourier_transform(field:(list, np.array), dx:float)->(list,np.array):
    if isinstance(field, list):
        ffield = [np.fft.rfft(f[1:-1]) for f in field]
        kx = 2*np.pi/(dx*(field[0].shape[0]-2))
        kfield = kx*np.arange(ffield[0].shape[0])
    else:
        ffield = np.fft.rfft(field[1:-1])
        kx = 2*np.pi/(dx*(field.shape[0]-2))
        kfield = kx*np.arange(ffield.shape[0])
    return ffield, kfield
                          

# Definition of Finite difference functions used to resolve convection-diffusion equation

## function used for initial condition
If/When you modify this function, don't forget to execute again the cell (press shift+enter for that)

In [ ]:
def initfield_1D(x: np.array):
    """
    generate initial profile for advdiff simulations,
    :param x: meshgrid for X values
    :return: 1D field
    """
    u0 = np.zeros(x.shape)
    u0 = np.exp(-(x-.5*x.max())**2/10)

    # exemple for gate
#     u0[:] = 1
#     u0[0:u0.shape[0]//4] = 0
#     u0[3*u0.shape[0] // 4:] = 0
    return u0

## definition of simulation's parameters

In [ ]:

# list of parameters used in both advdiff et H2D.
# Remark : the # starting line indicates a comment and is not necessary

# definition of parameter input to be used
"""
# time step
dt: .000001

# x step
dx:  .015

# y step (used only for H2D simulations)
dy: .01

# Points in X direction
Nx:  128

# Points in Y direction
Ny: 128

# modes in Y direction
Nm:  32

# wave number in Y direction
ky: .1

# end time
Tmax: .00001

# output time
Toutput: .000001

# diffusion coefficient
C:  .02

# advection coefficient
V:  -.2

# time scheme
# can be
# eule for euler explicit (default)
# euli for euler implicit
# RK2 for Runge-Kutta 2
# RK4 for Runge-Kutta 4
# CN for Cranck-Nicholson
scheme: eule



# first order derivatives
# fwd for forward [default] (u[i+1]-u[i])/dx
# bwd for backward (u[i]-u[i-1])/dx
# cent for centered (u[i+1]-u[i-1])/(2dx)
derivative = "fwd"

# boundary condition
# dir for Dirichlet u[0] = 0
# neu for Von Neumann du/dx[0] = 0
# per for periodic u[0] = u[Lx]
boundaries = "dir"

"""
;

# Section 0 : Introduction

Solve convection-diffusion equation : 
$$\Large \frac{\partial u(x,t)}{\partial t}+V\frac{\partial u(x,t)}{\partial x} = C\frac{\partial^2 u(x,t)}{\partial x^2}$$
$$\Large u(x, t=0) = u_0(x) $$
$$\Large u(0, t) = u(L_x, t) $$

<font size=4>
<p>First, we define parameters in a string which will then be used for the simulation. All the parameters are described in the upper cell.
This variable is converted to an usable dict through the function load_from_string.
    </p>
</font>

In [ ]:
params_ex_diff_str = """
C: 0.1
Nx: 1024
Tmax: 500
Toutput: 100 
V: -0.2
dt: 0.12
dx: 0.1
scheme: rk4
boundaries: per
derivative: bwd
"""
params_ex_diff = load_from_string(params_ex_diff_str)
display(params_ex_diff)

<font size=4> 
    To run a new simulation you just need to use the function simulate with the parameters written previously as argument
</font>

In [ ]:
time_output, output_ex_diff, time_exec = advdiff.simulate_1d(**params_ex_diff, verbose=False, init=initfield_1D)

In [ ]:
ffield, kx=fourier_transform(output_ex_diff , params_ex_diff['dx'])

In [ ]:
for f in ffield:
    plt.semilogy(kx, np.abs(f), '+')